In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
df1 = pd.read_csv("Sepsis_Cases_Log.csv")
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15099 entries, 0 to 15098
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Case ID                    15075 non-null  object 
 1   Activity                   15099 non-null  object 
 2   Complete Timestamp         15099 non-null  object 
 3   Variant                    15099 non-null  object 
 4   Variant index              15099 non-null  int64  
 5   lifecycle:transition       15099 non-null  object 
 6   org:group                  15099 non-null  object 
 7   InfectionSuspected         1013 non-null   object 
 8   DiagnosticBlood            1013 non-null   object 
 9   DisfuncOrg                 1013 non-null   object 
 10  SIRSCritTachypnea          1013 non-null   object 
 11  Hypotensie                 1013 non-null   object 
 12  SIRSCritHeartRate          1013 non-null   object 
 13  Infusion                   1013 non-null   obj

In [4]:
df1.head()

,Case ID,Activity,Complete Timestamp,Variant,Variant index,lifecycle:transition,org:group,InfectionSuspected,DiagnosticBlood,DisfuncOrg,...,SIRSCritLeucos,Oligurie,DiagnosticLacticAcid,Diagnose,Hypoxie,DiagnosticUrinarySediment,DiagnosticECG,Leucocytes,CRP,LacticAcid
0,A,ER Registration,15:41.0,Variant 61,61,complete,A,True,True,True,...,False,False,True,A,False,True,True,NaN,NaN,NaN
1,A,Leucocytes,27:00.0,Variant 61,61,complete,B,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.6,NaN,NaN
2,A,CRP,27:00.0,Variant 61,61,complete,B,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN
3,A,LacticAcid,27:00.0,Variant 61,61,complete,B,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2
4,A,ER Triage,33:37.0,Variant 61,61,complete,C,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
